In [ ]:
%pip install pandas
%pip install influxdb_client
%pip install cmdstanpy==0.9.68
%pip install pystan==2.19.1.1
%pip install seaborn
%pip install prophet

### Get data from influx db
based on https://github.com/influxdata/influxdb-client-python/blob/master/examples/

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

In [ ]:
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS
#read influx info from config file

with open('../server/influxconfig.json', 'r') as fd:
    config = json.load(fd)

#need to edit the query cause idk what i have to predict on

#from(bucket: "temp")
#    |> range(start: -24h)
#    |> filter(fn: (r) => r._measurement == "val" and r._field == "value" and r._value > -300 and r.sensor == string(v: ${Sensor_id}))


query_temp = 'from(bucket:"temp")' \
        ' |> range(start:2007-12-10T15:00:00Z, stop:2016-01-20T15:00:00Z)'\
        ' |> filter(fn: (r) => r._measurement == "views")' \
        ' |> filter(fn: (r) => r._field == "y")'

client =  InfluxDBClient(url='http://'+config.host+':'+config.port, token=config.token, org=config.org, debug=False)
write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()  
result = query_api.query(query_temp)

In [ ]:
for table in result:
    print(table)
    for record in table.records:
        print(record)
#i should be querying now, so this part will be done as soon as i get home

In [ ]:
#build the dataframe with the data needed for the algorithm, when you have fields
#ideally df['Time'], df['v']

In [ ]:
values = df['temp']
values.index = df['time']
values.plot(sublots=True)

In [ ]:
###stationary test
from statsmodels.tsa.stattools import adfuller
#split dataset
l = len(df.values)
cut = int(l*.8)
train = df[:cut]
test = df[cut:]
res = adfuller(df)
print('ADF val: ', res[0], 'p-value:', res[1])

In [ ]:
#if p-val < 0.05 then series is stationary and it's impossible to use autoregression models
#so i have to differentiate
train_diff = train.diff().dropna()
res = adfuller(train_diff)
print('ADF val: ', res[0], 'p-value:', res[1])


In [ ]:
#plot with points to see what is the stationary point as in the slides
#autocorrelation  -> find  that is the highest with the minimal difference from the deviation(?) 
from statsmodels.graphics import tsaplots
fig = tsaplots.plot_acf(train_diff, lags=10)
plt.show()

In [ ]:
#partial autocorrelation -> find q that is the highest with the minimal difference from the deviation(?) 
fig = tsaplots.plot_pacf(train_diff, lags=10)
plt.show()

In [ ]:
#once you have both p and q it is possibile to apply the ARIMA model
from statsmodels.tsa.arima_model import ARIMA
history = [x for x in train]
predictions = list()
print(test)
for t in test.index:
  model = ARIMA(history, order=(p,q,d)) #d is the number of time that the raw values are differentiated
  model_fit = model.fit()
  output = model_fit.forecast()
  yest = output[0]
  predictions.append(yest)
  obs= test[t]
  history.append(obs)
  print ('predicted=%f, expected=%f' % (yest, obs))

In [ ]:
#plt
import math
from sklearn.metrics import mean_squared_error

rmse = math.sqrt(mean_squared_error(test, predictions))
print('Test RMSE: %.3f'% rmse)

df2 = pd. DataFrame (predictions)
df2. set_index(test.index, inplace=True)
plt.plot(test)
plt.plot(df2, color='red')
plt.show( )